In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

In [22]:
X_train = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_train_all.csv')

X_test = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_test_all.csv')

y_train = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_train_all.csv")

y_test = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_test_all.csv")

In [11]:
y_train["0"].nunique()

10

In [14]:
image_height = 48
image_width = 48
num_channels = 1
num_classes = 10
num_epochs = 10
batch_size = 64

In [23]:
# Reshape X_train to match the expected input shape
X_train_reshaped = np.reshape(
    X_train, (X_train.shape[0], image_height, image_width, num_channels))

# Reshape X_test to match the expected input shape
X_test_reshaped = np.reshape(
    X_test, (X_test.shape[0], image_height, image_width, num_channels))

# Convert y_train to one-hot encoded format
y_train_encoded = to_categorical(y_train, num_classes)

# Convert y_test to one-hot encoded format
y_test_encoded = to_categorical(y_test, num_classes)

In [18]:
X_train_reshaped.shape

(9690, 48, 48, 1)

In [17]:
y_train_encoded.shape

(9690, 10)

In [31]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_encoded,
          epochs=num_epochs, batch_size=batch_size)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

Epoch 1/10


2023-11-23 12:47:58.049793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 3s 17ms/step - loss: 8.5633 - accuracy: 0.6021
Epoch 2/10
152/152 [==============================] - 2s 15ms/step - loss: 0.3394 - accuracy: 0.9052
Epoch 3/10
152/152 [==============================] - 2s 15ms/step - loss: 0.2178 - accuracy: 0.9451
Epoch 4/10
152/152 [==============================] - 2s 15ms/step - loss: 0.1475 - accuracy: 0.9598
Epoch 5/10
152/152 [==============================] - 2s 15ms/step - loss: 0.0961 - accuracy: 0.9747
Epoch 6/10
152/152 [==============================] - 2s 15ms/step - loss: 0.0961 - accuracy: 0.9769
Epoch 7/10
152/152 [==============================] - 2s 16ms/step - loss: 0.0798 - accuracy: 0.9814
Epoch 8/10
152/152 [==============================] - 2s 15ms/step - loss: 0.0896 - accuracy: 0.9779
Epoch 9/10
152/152 [==============================] - 2s 15ms/step - loss: 0.0728 - accuracy: 0.9826
Epoch 10/10
152/152 [==============================] - 2s 15ms/step - loss: 0.0818 - accuracy: 0.9806


In [32]:
model.evaluate(X_train_reshaped, y_train_encoded)

 11/303 [>.............................] - ETA: 3s - loss: 0.0362 - accuracy: 0.9773

2023-11-23 12:48:23.903036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


303/303 [==============================] - 3s 10ms/step - loss: 0.0721 - accuracy: 0.9778


[0.0720549076795578, 0.9778121709823608]

In [33]:
model.evaluate(X_test_reshaped, y_test_encoded)

97/97 [==============================] - 1s 11ms/step - loss: 1.2403 - accuracy: 0.8744


[1.2403024435043335, 0.8744336366653442]

In [ ]:
# 97% accuracy on the test set

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_encoded, epochs=num_epochs, batch_size=batch_size)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Epoch 1/10


2023-11-23 12:43:08.003154: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 5s 25ms/step - loss: 2.0460 - accuracy: 0.5852
Epoch 2/10
152/152 [==============================] - 4s 24ms/step - loss: 0.1958 - accuracy: 0.9531
Epoch 3/10
152/152 [==============================] - 4s 24ms/step - loss: 0.0749 - accuracy: 0.9812
Epoch 4/10
152/152 [==============================] - 4s 23ms/step - loss: 0.0736 - accuracy: 0.9806
Epoch 5/10
152/152 [==============================] - 4s 23ms/step - loss: 0.0523 - accuracy: 0.9878
Epoch 6/10
152/152 [==============================] - 4s 24ms/step - loss: 0.0371 - accuracy: 0.9912
Epoch 7/10
152/152 [==============================] - 4s 23ms/step - loss: 0.0159 - accuracy: 0.9967
Epoch 8/10
152/152 [==============================] - 4s 23ms/step - loss: 0.0020 - accuracy: 0.9999
Epoch 9/10
152/152 [==============================] - 3s 23ms/step - loss: 6.8834e-04 - accuracy: 1.0000
Epoch 10/10
152/152 [==============================] - 3s 23ms/step - loss: 4.2156e-04 - accuracy:

In [28]:
from keras.layers import Dropout

# 98% accuracy on the test set

# Define the CNN model with dropout layers
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Add dropout layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train_encoded, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10


2023-11-23 12:45:23.095324: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 5s 29ms/step - loss: 3.1158 - accuracy: 0.2752
Epoch 2/10
152/152 [==============================] - 4s 26ms/step - loss: 1.2745 - accuracy: 0.5422
Epoch 3/10
152/152 [==============================] - 4s 26ms/step - loss: 0.4843 - accuracy: 0.8583
Epoch 4/10
152/152 [==============================] - 4s 25ms/step - loss: 0.2470 - accuracy: 0.9289
Epoch 5/10
152/152 [==============================] - 4s 26ms/step - loss: 0.1831 - accuracy: 0.9481
Epoch 6/10
152/152 [==============================] - 4s 27ms/step - loss: 0.1338 - accuracy: 0.9632
Epoch 7/10
152/152 [==============================] - 4s 27ms/step - loss: 0.1227 - accuracy: 0.9677
Epoch 8/10
152/152 [==============================] - 4s 26ms/step - loss: 0.0865 - accuracy: 0.9774
Epoch 9/10
152/152 [==============================] - 4s 27ms/step - loss: 0.0723 - accuracy: 0.9777
Epoch 10/10
152/152 [==============================] - 4s 27ms/step - loss: 0.0846 - accuracy: 0.9791


In [29]:
model.evaluate(X_train_reshaped, y_train_encoded)

  5/303 [..............................] - ETA: 3s - loss: 0.0764 - accuracy: 0.9688 

2023-11-23 12:46:06.961695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


303/303 [==============================] - 4s 12ms/step - loss: 0.0162 - accuracy: 0.9956


[0.01616240292787552, 0.9955624341964722]

In [30]:
model.evaluate(X_test_reshaped, y_test_encoded)

97/97 [==============================] - 1s 12ms/step - loss: 0.0591 - accuracy: 0.9832


[0.059141576290130615, 0.9831715226173401]

### Here we got 98.3% accuracy on the test set